In [11]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json

In [12]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [13]:
# URL to find information on Coronavirus
url = 'https://www.dshs.state.tx.us/news/updates.shtm#coronavirus'
url2 = "http://www.google.com"
url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus"

In [14]:
# Visiting the url 
browser.visit(url)

In [15]:
# Creating the beautiful soup to find the tables we need 
html = browser.html
soup = bs(html, 'html.parser')

In [16]:
# All tables on this webpage, this might be revised later
tables = soup.find_all('table')

In [17]:
# Stats for total cases  
total_table = tables[1]
totals = total_table.find_all('td')
total_cases = totals[0].text
total_deaths = totals[1].text

In [18]:
df_stats = pd.DataFrame({"Cases":total_cases, "Deaths":total_deaths}, index=[0])
df_stats
df_stats.to_csv('static/csv/stats.csv', index=False)

In [19]:
browser.visit(url3)

In [20]:
harris_table = browser.find_by_id('covidtable')

In [21]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [22]:
harris_soup

<html lang="en-US" style="height: 100%;"><head id="Head">
<!--*********************************************-->
<!-- DNN Platform - http://www.dnnsoftware.com   -->
<!-- Copyright (c) 2002-2018, by DNN Corporation -->
<!--*********************************************-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="F0PqTcRKcnKG7nlC0FL6ptMucbfv5jV5cOPHE7k50sw" name="google-site-verification"/><title>
	2019 Novel Coronavirus
</title><meta content=",DotNetNuke,DNN" id="MetaKeywords" name="KEYWORDS"/><meta content="DotNetNuke " id="MetaGenerator" name="GENERATOR"/><meta content="INDEX, FOLLOW" id="MetaRobots" name="ROBOTS"/><link href="/Resources/Shared/stylesheets/dnndefault/7.0.0/default.css?cdv=1103" rel="stylesheet" type="text/css"/><link href="/Resources/Search/SearchSkinObjectPreview.css?cdv=1103" rel="stylesheet" type="text/css"/><link href="/Portals/27/skins/hcph/skin.css?cdv=1103" rel="stylesheet" type="text/css"/><link href="/Portals/27/Contai

In [23]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [24]:
columns = ['Sex', 'Location', 'Age_Range', 'Status']


In [25]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [26]:
harris_df['Sex'] = harris_df['Sex'].apply(lambda x : x.split()[2])

In [27]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [28]:
harris_df.head()

,Sex,Location,Age_Range,Status
0,Female,SW,60-69,Recovered
1,Male,SW,60-69,Recovered
2,Male,NW,60-69,Recovered
3,Female,NW,60-69,Recovered
4,Female,SW,20-29,Recovered


In [29]:
# Selecting the table with all the Texas Counties
TX_counties = tables[2]
TX_counties = TX_counties.find_all('td')

In [30]:
# Creating lists of the counties and current number of cases per county

counties = []
cases = []

counter = 0 
for county in TX_counties:
    if counter % 2 == 0:
        counties.append(county.text)
        
    elif counter % 2 != 0:
        cases.append(county.text)
        
    counter = counter + 1
    

In [31]:
# Creating the first data frame. 
df = pd.DataFrame(list(zip(counties, cases)), columns =['Texas_County', 'COVID_19_Cases'])
df

,Texas_County,COVID_19_Cases
0,Bell,3
1,Bexar,24
2,Bowie,1
3,Brazoria,12
4,Brazos,2
5,Brown,1
6,Cameron,6
7,Castro,2
8,Chambers,1
9,Collin,24


In [32]:
# Going to collect lat long information on these Texas Counties. 
counties = df.Texas_County.tolist()
counties.pop()
counties

['Bell',
 'Bexar',
 'Bowie',
 'Brazoria',
 'Brazos',
 'Brown',
 'Cameron',
 'Castro',
 'Chambers',
 'Collin',
 'Comal',
 'Crane',
 'Dallas',
 'Deaf Smith',
 'Denton',
 'DeWitt',
 'El Paso',
 'Ellis',
 'Falls',
 'Fannin',
 'Fort Bend',
 'Galveston',
 'Gregg',
 'Harris',
 'Hays',
 'Hidalgo',
 'Hockley',
 'Johnson',
 'Lavaca',
 'Lubbock',
 'Matagorda',
 'McLennan',
 'Medina',
 'Midland',
 'Montgomery',
 'Nueces',
 'Oldham',
 'Parker',
 'Potter',
 'Rusk',
 'Smith',
 'Tarrant',
 'Travis',
 'Webb',
 'Wichita',
 'Williamson']

In [33]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [34]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [35]:
Counties_with_Cases = []

for i in counties:
    
    if i in All_Tex_Counties:
        Counties_with_Cases.append(i)
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Bell---found
Bexar---found
Bowie---found
Brazoria---found
Brazos---found
Brown---found
Cameron---found
Castro---found
Chambers---found
Collin---found
Comal---found
Crane---found
Dallas---found
Deaf Smith---found
Denton---found
DeWitt---found
El Paso---found
Ellis---found
Falls---found
Fannin---found
Fort Bend---found
Galveston---found
Gregg---found
Harris---found
Hays---found
Hidalgo---found
Hockley---found
Johnson---found
Lavaca---found
Lubbock---found
Matagorda---found
McLennan---found
Medina---found
Midland---found
Montgomery---found
Nueces---found
Oldham---found
Parker---found
Potter---found
Rusk---found
Smith---found
Tarrant---found
Travis---found
Webb---found
Wichita---found
Williamson---found


In [36]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [37]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Bell,-97.477908,31.037783,14,48027,2.178762,0.265927,"(-97.477908, 31.037783)"
Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
Bowie,-94.423243,33.445728,19,48037,4.057053,0.231733,"(-94.423243, 33.445728)"
Brazoria,-95.455785,29.193454,20,48039,8.449999,0.342870,"(-95.455785, 29.193454)"
Brazos,-96.302294,30.661043,21,48041,2.811591,0.143907,"(-96.302294, 30.661043)"


In [38]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "Texas_County"})
Geo_df

,Texas_County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Bell,-97.477908,31.037783,14,48027,2.178762,0.265927,"(-97.477908, 31.037783)"
1,Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
2,Bowie,-94.423243,33.445728,19,48037,4.057053,0.231733,"(-94.423243, 33.445728)"
3,Brazoria,-95.455785,29.193454,20,48039,8.449999,0.342870,"(-95.455785, 29.193454)"
4,Brazos,-96.302294,30.661043,21,48041,2.811591,0.143907,"(-96.302294, 30.661043)"
5,Brown,-98.999896,31.774323,25,48049,2.082173,0.236004,"(-98.999896, 31.774323)"
6,Cameron,-97.515000,26.138477,31,48061,6.776491,0.237848,"(-97.515, 26.138477)"
7,Castro,-102.261761,34.530003,35,48069,1.923921,0.228709,"(-102.261761, 34.530003)"
8,Chambers,-94.608913,29.739123,36,48071,3.954637,0.152678,"(-94.608913, 29.739123)"
9,Collin,-96.572489,33.187891,43,48085,1.931216,0.221922,"(-96.572489, 33.187891)"


In [39]:
result = pd.merge(df, Geo_df, on='Texas_County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,Texas_County,COVID_19_Cases,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Bell,3,-97.477908,31.037783,14,48027,2.178762,0.265927,"(-97.477908, 31.037783)"
1,Bexar,24,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
2,Bowie,1,-94.423243,33.445728,19,48037,4.057053,0.231733,"(-94.423243, 33.445728)"
3,Brazoria,12,-95.455785,29.193454,20,48039,8.449999,0.342870,"(-95.455785, 29.193454)"
4,Brazos,2,-96.302294,30.661043,21,48041,2.811591,0.143907,"(-96.302294, 30.661043)"


In [40]:
result = result[['Texas_County', 'COVID_19_Cases', 'lat', 'long']]
result

,Texas_County,COVID_19_Cases,lat,long
0,Bell,3,31.037783,-97.477908
1,Bexar,24,29.448845,-98.519663
2,Bowie,1,33.445728,-94.423243
3,Brazoria,12,29.193454,-95.455785
4,Brazos,2,30.661043,-96.302294
5,Brown,1,31.774323,-98.999896
6,Cameron,6,26.138477,-97.515000
7,Castro,2,34.530003,-102.261761
8,Chambers,1,29.739123,-94.608913
9,Collin,24,33.187891,-96.572489


In [41]:
# Scrape the Lat Long data

# lats = []
# longs = []
# for county in counties:
#     browser.visit(url2)
    
#     browser.fill('q', f'{county} county Texas Lattitude Longitude')
    
#     time.sleep(2)

#     enter = browser.find_by_name('q')
    
#     enter.type(Keys.RETURN)
    
#     time.sleep(2)
    
#     geo_info = browser.find_by_css('.Z0LcW').text
    
#     lat = geo_info.split('°')[0]
#     long = geo_info.split(',')[1].split('°')[0].strip()
    
#     lats.append(float(lat))
#     longs.append(-float(long))
    
    
    
    

In [42]:
Counties_Pending = df.tail(1)

In [43]:
Counties_Pending

,Texas_County,COVID_19_Cases
46,Pending County Assignment,65


In [44]:
df.drop(df.tail(1).index,inplace=True)

In [45]:
# # Adding lat and longs to the dataframe
# df['lat'] = lats
# df['long'] = longs

In [46]:
# df.head()

In [47]:
browser.quit()

In [48]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['Texas_County', 'COVID_19_Cases']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [49]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)